# Data exploration

## Load paths

In [ ]:
def umlaut(string):
    if not isinstance(string, str):
        raise TypeError("The oject is a " + str(type(string)) + ", not a string!")
    string = string.replace("Ã¼", "ü")
    string = string.replace("Ã¤", "ä")    
    string = string.replace("Ã¶", "ö")    
    string = string.replace("Ãœ", "Ü")    
    string = string.replace("Ã„", "Ä")    
    string = string.replace("Ã–", "Ö")    
    string = string.replace("ÃŸ", "ß")    
    return string

def replace_umlauts(dictionary):
    try:
        dictionary = umlaut(dictionary)
    except TypeError:
        copy = dictionary
        try:
            iteritems = dictionary.items()
        except AttributeError:
            iteritems = enumerate(dictionary)
        for k, v in iteritems:
            copy[k] = replace_umlauts(v)
        dictionary = copy
    return dictionary

In [ ]:
import json

with open('paths.json') as f:
    data = json.load(f)
    data = replace_umlauts(data)

data

In [ ]:
keywordFile = data['keywords']['file']
keywordFile

In [ ]:
dataDirs = data['data']['directory']
dataDirs

In [ ]:
testFile = data['data']['test']
testFile

## Load data

### Keywords
Keywords are in a table in a MS word document

In [ ]:
# read the first table of the docx file

from docx import Document
document = Document(keywordFile)
table = document.tables[0]
text = []
for i in range(len(table.rows)):
    text.append([])
    for j in range(len(table.columns)):
        text[i].append(table.rows[i].cells[j].text)
        
# remove header
text = text[1:]

In [ ]:
# split in English and German
text_en = list(map(lambda x: x[2:4], text))
text_de = list(map(lambda x: x[0:2], text))
text_de[0]

In [ ]:
def produce_dict(text):
    dictionary = {}
    i = 0
    key = ''
    for row in text:
        word = row[0]
        veto = row[1]
        if word != '':
            key = word
            dictionary[key] = []
        if veto != '':
            dictionary[key].append(veto)
    return dictionary

In [ ]:
# make dictionary of pairwise keywordsusions
keywords_de = produce_dict(text_de)
keywords_en = produce_dict(text_en)
keywords_de[next(iter(keywords_de))]

### Documents

Documents are drawn from URLs in MS Outlook files

In [ ]:
import win32com.client
import os
import re
from collections import OrderedDict

outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

URLs = []
maxMails = 1
openMails = 0
for directory in dataDirs:
    for filename in os.listdir(directory):
        openMails += 1
        if openMails > maxMails:
            break
        filePath = os.path.abspath(os.path.join(directory, filename))
        print(filePath)
        msg = outlook.OpenSharedItem(filePath)
#         print(msg.SenderName)
#         print(msg.Body)
        matchObj = re.findall(r'<(.*?)>', msg.Body)
        matchObj = list(OrderedDict.fromkeys(matchObj))  # remove duplicates
        matchObj = [link for link in matchObj if not 'alerts' in link]
#         matchObj.remove('Als irrelevant markieren ')
        if matchObj:
            print("Success! Found " + str(len(matchObj)) + " links")
#             pprint("matchObj : ", matchObj)
            for index, element in enumerate(matchObj):
                print(str(index) + ": " + element)
#                 result = re.search('url=(.*?)&', element)
#                 print(result.group(1))
#                 print(element)

        else:
            print("No match!")

msg = outlook.OpenSharedItem(os.path.abspath(os.path.join(dataDirs[0], testFile)))

In [ ]:
from bs4 import BeautifulSoup


In [ ]:
print(os.path.abspath(os.path.join(dataDirs[0], testFile)))
msg = outlook.OpenSharedItem(os.path.abspath(os.path.join(dataDirs[0], testFile)))